In [ ]:
pip install transformers torch flask


In [ ]:
import pandas as pd
import numpy.core.multiarray

# Convert to CSV
df = pd.read_parquet('validation-00000-of-00001.parquet')


In [ ]:
df

,context,question,answers
0,Mrs. Wetterauer is a 54-year-old female with c...,Has the patient ever taken glyburide for their...,"{'answer_end': [1578], 'answer_start': [1553],..."
1,Ms. Heit is a 67-year-old female who received ...,has there been a prior protonix,"{'answer_end': [432], 'answer_start': [356], '..."
2,Marcelo Walts was admitted to the medical serv...,Was the patient ever given medication for whee...,"{'answer_end': [1244], 'answer_start': [1186],..."
3,A 79-year-old male with history of non-insulin...,What medication has the patient take for dvt p...,"{'answer_end': [1861], 'answer_start': [1810],..."
4,Mr. Sheumaker is a 65-year-old gentleman with ...,Why was the patient prescribed percocet,"{'answer_end': [412], 'answer_start': [328], '..."
...,...,...,...
32734,This 54 year old gentleman presented to the Wi...,has the patient had azmacort,"{'answer_end': [2358], 'answer_start': [2329],..."
32735,"This 70-year-old woman with no known CAD, card...",has there been a prior zestril ( lisinopril ),"{'answer_end': [1090], 'answer_start': [1057],..."
32736,Ms. Dozois is a 64-year-old female admitted to...,has the patient had red blood transfusions,"{'answer_end': [874], 'answer_start': [805], '..."
32737,This 63 year-old male with a history of periph...,Why did the patient have coumadin,"{'answer_end': [1918], 'answer_start': [1855],..."


In [ ]:
df['answers'][1]

{'answer_end': array([432]),
 'answer_start': array([356]),
 'text': array(['daily, CellCept 1500 mg b.i.d., Protonix 20 mg daily, Pravachol 40 mg daily,'],
       dtype=object)}

In [ ]:
import pandas as pd
from transformers import BertTokenizerFast
import torch

tokenizer = BertTokenizerFast.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

def preprocess_emrqa_dataset(df, tokenizer, max_len=512):
    processed_data = []

    for index, row in df.iterrows():
        context = row['context']
        question = row['question']
        answers = row['answers']

        answer_start = answers['answer_start'][0]
        answer_end = answers['answer_end'][0]
        answer_text = context[answer_start:answer_end]

        encoding = tokenizer.encode_plus(
            question, context,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_offsets_mapping=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        offsets = encoding['offset_mapping'].squeeze().tolist()

        start_token = end_token = None
        for i, offset in enumerate(offsets):
            if offset[0] == answer_start:
                start_token = i
            if offset[1] == answer_end:
                end_token = i

        if start_token is None or end_token is None:
            continue

        processed_data.append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': torch.tensor(start_token),
            'end_positions': torch.tensor(end_token),
            'context': context,
            'question': question,
            'answer': answer_text
        })

    return processed_data

processed_dataset = preprocess_emrqa_dataset(df, tokenizer)

torch.save(processed_dataset, 'processed_emrqa_dataset.pt')

print(f"Processed {len(processed_dataset)} entries.")


Processed 25813 entries.


In [ ]:
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

processed_data = torch.load('processed_emrqa_dataset.pt')

input_ids = torch.stack([item['input_ids'] for item in processed_data])
attention_masks = torch.stack([item['attention_mask'] for item in processed_data])
start_positions = torch.stack([item['start_positions'] for item in processed_data])
end_positions = torch.stack([item['end_positions'] for item in processed_data])

dataset = TensorDataset(input_ids, attention_masks, start_positions, end_positions)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=16)


<ipython-input-9-6439eb034aff>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  processed_data = torch.load('processed_emrqa_dataset.pt')


In [ ]:
from transformers import BertForQuestionAnswering, AdamW

# Load the BERT model for Question Answering
model = BertForQuestionAnswering.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

epochs = 3
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')

    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_start_positions = batch[2].to(device)
        b_end_positions = batch[3].to(device)

        model.zero_grad()

        outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                        start_positions=b_start_positions, end_positions=b_end_positions)

        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()

        optimizer.step()

    print(f"Total training loss: {total_loss / len(train_dataloader)}")

    model.eval()
    eval_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_start_positions = batch[2].to(device)
            b_end_positions = batch[3].to(device)

            outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                            start_positions=b_start_positions, end_positions=b_end_positions)

            eval_loss += outputs.loss.item()

    print(f"Validation loss: {eval_loss / len(val_dataloader)}")


Epoch 1/3


  0%|          | 0/1291 [00:00<?, ?it/s]